In [2]:
from importlib import reload
import load_data
reload(load_data)
from load_data import *

In [6]:
url = "https://datasets.imdbws.com/"
filenames = ["name.basics.tsv.gz",
             "title.akas.tsv.gz",
             "title.basics.tsv.gz",
             "title.crew.tsv.gz",
             "title.episode.tsv.gz",
             "title.principals.tsv.gz",
             "title.ratings.tsv.gz"
]

output_paths = download_file(url, filenames, "data/")

# Pierwszy rzut oka na dane

Pierwsza ramka:

In [ ]:
path_to_file = "data/name.basics.tsv.gz"
df = load_to_dataframe(path_to_file).replace(to_replace=r"\N", value=pd.NA)

In [ ]:
df.head()

In [8]:
df.shape

(13583187, 6)

In [9]:
df.columns

Index(['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession',
       'knownForTitles'],
      dtype='object')

Druga ramka:

In [3]:
path_to_file = "data/title.akas.tsv.gz"
df = load_to_dataframe(path_to_file).replace(to_replace=r"\N", value=pd.NA)

In [4]:
df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Carmencita,<NA>,<NA>,original,<NA>,1
1,tt0000001,2,Carmencita,DE,<NA>,<NA>,literal title,0
2,tt0000001,3,Carmencita,US,<NA>,imdbDisplay,<NA>,0
3,tt0000001,4,Carmencita - spanyol tánc,HU,<NA>,imdbDisplay,<NA>,0
4,tt0000001,5,Καρμενσίτα,GR,<NA>,imdbDisplay,<NA>,0


In [5]:
df.shape

(48657469, 8)

In [6]:
df.columns

Index(['titleId', 'ordering', 'title', 'region', 'language', 'types',
       'attributes', 'isOriginalTitle'],
      dtype='object')

In [36]:
ambig_index = df.loc[:, ['titleId', 'title', 'region', 'isOriginalTitle']].query(
    "isOriginalTitle == 0"
).merge(
    df.loc[:, ['titleId', 'title', 'isOriginalTitle']].query("isOriginalTitle == 1"),
    on="titleId",
    how="inner",
    suffixes=("", "_y")
).query(
    "title == title_y"
).drop(
    labels=["title_y"],
    axis=1
).loc[:, ["titleId", "region"]].groupby(
    by=["titleId"]
).count().where(
    lambda region: region > 1
).dropna().index

Trzecia ramka:

In [39]:
path_to_file = "data/title.basics.tsv.gz"
df = load_to_dataframe(path_to_file)

c:\Users\priva\OneDrive\Desktop\STUDIA\Narzędzia Informatyczne w Pythonie wspierające Analizę Danych\Projekt zaliczeniowy\load_data.py:22: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path_to_file,


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [40]:
df.replace(to_replace=r"\N", value=pd.NA).head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,<NA>,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,<NA>,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,<NA>,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,<NA>,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,<NA>,1,"Comedy,Short"


Czwarta ramka:

In [ ]:
path_to_file = "data/title.crew.tsv.gz"
df = load_to_dataframe(path_to_file).replace(to_replace=r"\N", value=pd.NA)
df.head()

Piąta ramka:

In [ ]:
path_to_file = "data/title.episode.tsv.gz"
df = load_to_dataframe(path_to_file).replace(to_replace=r"\N", value=pd.NA)
df.head()

Szósta ramka:

In [ ]:
path_to_file = "data/title.principals.tsv.gz"
df = load_to_dataframe(path_to_file).replace(to_replace=r"\N", value=pd.NA)
df.head()

Siódma ramka:

In [ ]:
path_to_file = "data/title.ratings.tsv.gz"
df = load_to_dataframe(path_to_file).replace(to_replace=r"\N", value=pd.NA)
df.head()